In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

# Описание проектной задачи

Распознавание чисел на изображениях — это задача многоклассовой классификации. В качестве объекта, для которого необходимо предсказать класс, выступает изображение. Возможных классов столько, сколько в данных представлено чисел, которые необходимо научиться распознавать.

В качестве данных для решения задачи будут использоваться изображения из базы данных MNIST — классического набора данных для создания и проверки качества работы алгоритмов компьютерного зрения.

MNIST состоит из чёрно-белых изображений рукописных цифр. В базе представлены изображения цифр от 0
до 9. Каждое изображение имеет разрешение 28×28 пикселей.

В данных, которые будут использоваться для решения задачи, представлено 42 000 изображений. Это подмножество всех данных, доступных в MNIST.

In [ ]:
data = pd.read_csv